# Prepare data

In [1]:
from script.func_split_data import split_data

In [2]:
split_df = split_data(data_path='Cosmenet_product_20231018', data_csv='datas_20231018.csv')
split_df.split_data()
split_df.report_train_test_split()
print()
split_df.report_train_test_val_split()

amount of all data : 60196
amount of all class : 4178
amount of data 2-8 img : 1548
amount of 2-8 img class : 278
amount of data more 8 img : 58631
amount of more 8 img class : 3883
amount of data & class only one : 17

amount of train split : 38474
amount of train split class : 3184
amount of test split : 9620
amount of test split class : 3184
amount of train val : 8430
amount of train val class : 699
amount of test val : 2107
amount of test val class : 699
amount of train val mix : 9204
amount of train val mix class : 977
amount of test val mix : 2881
amount of test val mix class : 977


In [3]:
df_train_split, df_test_split = split_df.get_train_test()
df_train_val_mix, df_test_val_mix = split_df.get_validate()

In [5]:
df_train_split.head(1)

,file_names,labels,images_path
0,11596_2.jpg,11596,/app/nfs_clientshare/Datasets/Cosmenet_product...


# extract and put to elasticsearch

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd

import torch

from script.tool import ROOT_NFS_TEST, standardize_feature
from tqdm.notebook import tqdm

import time

In [ ]:
def select_transformers_model(model, processor, pretrain="google/vit-base-patch16-224-in21k"):
    model = model.from_pretrained(pretrain)
    processor = processor.from_pretrained(pretrain)
    return model, processor

In [ ]:
# pipeline for transformer library
class pipeline_transformer:
    def __init__(self, layer, row=False, device='cuda:0'):
        self.device = device
        self.layer = layer
        self.row = row
    
    def selct_model(self, model, processor):
        self.model = model
        self.processor = processor
        self.model.eval().to(self.device)
    
    def process_model(self, img):
        inputs = self.processor(images=img, return_tensors="pt").to(self.device)
        outputs = self.model(**inputs)
        return outputs
        
    def extract(self, img):
        ### return specific layer
        outputs = self.process_model(img)
        if type(self.row) == bool and not self.row:
            outputs = outputs[self.layer]
        else:
            outputs = outputs[self.layer][:, self.row]
        outputs = outputs.flatten().unsqueeze(0)
        outputs = standardize_feature(outputs).to('cpu').detach().numpy()
        return outputs
    
    def report_test(self):
        img = Image.new('RGB', (224, 224))
        start_time_torch = time.time()
        outputs = self.process_model(img)
        delta_time_torch = time.time() - start_time_torch
        print("runtime :", delta_time_torch*1000, "ms")
        print(f"outputs layers : {outputs.keys()}")
        print(f"shape last_hidden_state : {outputs.last_hidden_state.shape}")
        print(f"shape pooler_output : {outputs.pooler_output.shape}")

In [ ]:
from transformers import ViTImageProcessor, ViTModel

In [ ]:
device = torch.device('cpu')

model, preprocess = select_transformers_model(ViTModel, ViTImageProcessor, 
                                              pretrain=ROOT_NFS_TEST / 'weights/vit_gg_lr2e-05_eu_9ep_0_95099acc')
vit_gg_trained_lr2e_05_pipe = pipeline_transformer(layer="last_hidden_state", row=0, device=device)
vit_gg_trained_lr2e_05_pipe.selct_model(model, preprocess)
vit_gg_trained_lr2e_05_pipe.report_test()

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es = Elasticsearch(HOST="http://localhost", PORT=9200)

In [ ]:
def check_index_exist(name="test_product"):
    if es.indices.exists(index=name):
            print(f"index {name} already exists")
    else:
        body_product = {
            "mappings":{
                "properties":{
                    "tag":{
                        "type":"keyword"
                    },
                    "labels":{
                        "type":"keyword"
                    },
                    "file_names":{
                        "type":"text"
                    },
                    "images_path":{
                        "type":"text"
                    },
                    "features":{  
                        "type":"dense_vector",
                        "dims":768,
                        "index":True,
                        "similarity": "dot_product"
                    },
                    "id":{
                        "type":"keyword"
                    }
                }
            }
        }
        err = es.indices.create(index=name, body=body_product)
        print(err)

In [ ]:
name_product = "vit_gg_lr2e_05_eu_9ep_0_95099acc"
check_index_exist(name_product)

In [ ]:
def to_unit_len(vector):
    return vector / np.linalg.norm(vector)

In [ ]:
def put_to_es(mode, dataframe, index_name, tag="train_split"):
    for n, img_path in enumerate(tqdm(dataframe['images_path'])):
        img = Image.open(img_path).convert('RGB')
        output = mode.extract(img).flatten()
        data = {
            "tag": tag,
            "labels": dataframe['labels'].iloc[n],
            "file_names": dataframe['file_names'].iloc[n],
            "images_path": img_path,
            "features": to_unit_len(output),
            "id": index_name+"_"+str(n)
        }
        es.index(index=index_name, id=index_name+"_"+str(n), body=data)

In [ ]:
put_to_es(vit_gg_trained_lr2e_05_pipe, df_train_split, name_product, tag="train_split")
put_to_es(vit_gg_trained_lr2e_05_pipe, df_test_split, name_product, tag="test_split")
put_to_es(vit_gg_trained_lr2e_05_pipe, df_train_val_mix, name_product, tag="train_val")
put_to_es(vit_gg_trained_lr2e_05_pipe, df_test_val_mix, name_product, tag="test_val")

# query score

In [ ]:
# triple loss
best_validation_loss = 100
best_acc_top = 0
best_acc_top_n = 0

for epoch in tqdm(range(EPOCHS), desc="Epochs"):
    running_loss = []
    for step, (anchor_img, positive_img, negative_img) in enumerate(tqdm(train_dl, desc="Training", leave=False)):
        anchor_out = cvt_feature_vit_gg.process_extract(anchor_img)
        positive_out = cvt_feature_vit_gg.process_extract(positive_img)
        negative_out = cvt_feature_vit_gg.process_extract(negative_img)
        
        loss = criterion(anchor_out, positive_out, negative_out)
        
        Optimizer.zero_grad()
        loss.backward()
        Optimizer.step()
        running_loss.append(loss.cpu().detach().numpy())
    
    x_trans = convert_feature_transformer(vit_gg, processor_vit_gg, layer="last_hidden_state", row=0, device=DEVICE)
    validation_loss = validate(x_trans)
    acc_top = 
    acc_top_n = 
    
    if acc_top > best_acc_top:
        best_acc_top = acc_top
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at Best acc top: {:.4f}".format(best_acc_top))
        
    if acc_top_n > best_acc_top_n:
        best_acc_top_n = acc_top_n
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at Best acc top n: {:.4f}".format(best_acc_top_n))
    
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss  # Update the best performance
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at Best performance: {:.4f}".format(best_validation_loss))
    
    if epoch%10 == 9:
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at epoch {}".format(epoch+1))

    print("Epoch: {}/{} — Loss: {:.4f} — validation_loss : {:.4f} — acc_top : {} — acc_top_n : {}".format(
        epoch+1, EPOCHS, np.mean(running_loss), validation_loss, acc_top, acc_top_n))